In [2]:
import pandas as pd

data = pd.read_excel("final_student_embeddings_batched.xlsx")
low = data[data["label"] == 1]
med = data[data["label"] == 2]
high = data[data["label"] == 3]

print(len(low))
print(len(med))
print(len(high))

452
1131
1728


In [3]:
!pip install -U imbalanced-learn



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:

import pandas as pd
from imblearn.over_sampling import SMOTE
# Drop non-numeric columns (like 'Student' and 'Teacher')
X = data.drop(columns=['Student', 'Teacher'], errors='ignore')

# Ensure there are no missing values
X = X.dropna()

# Convert to numeric data (if necessary)
X = X.apply(pd.to_numeric, errors='coerce')

# Ensure there are no NaN values after conversion
X = X.fillna(0)

# Target column
y = data['label']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create DataFrame from resampled data
X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
y_resampled_df = pd.DataFrame(y_resampled, columns=['Label'])

# Combine into one DataFrame
balanced_df = pd.concat([X_resampled_df, y_resampled_df], axis=1)

# Save to Excel
balanced_df.to_excel("Upsampled_bert.xlsx", index=False)
print("Upsampled Bert.")


Upsampled Bert.


In [21]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer
from sklearn.pipeline import make_pipeline

# Optional: XGBoost
try:
    from xgboost import XGBClassifier
    xgb_enabled = True
except ImportError:
    xgb_enabled = False

# Load data
data_df = pd.read_excel("Upsampled_Mpnet.xlsx")
target_col = "label"

# Detect embedding columns
first_col = data_df.columns[0]
embedding_columns = list(range(768)) if isinstance(first_col, int) else [str(i) for i in range(768)]
X_features = data_df[embedding_columns].values
y_labels = data_df[target_col]
le = LabelEncoder()
y_encoded_labels = le.fit_transform(y_labels)

# Apply NCA
nca_model = NeighborhoodComponentsAnalysis(n_components=2, random_state=42)
X_nca_transformed = nca_model.fit_transform(X_features, y_encoded_labels)

# Print top NCA contributors
top_features_to_show = 10
nca_components = nca_model.components_
for comp_idx in range(2):
    weights = nca_components[comp_idx]
    top_indices = np.argsort(np.abs(weights))[::-1][:top_features_to_show]
    print(f"\nTop {top_features_to_show} features for NCA Component {comp_idx + 1}:")
    for i in top_indices:
        print(f"Feature {i}: Weight = {weights[i]:.4f}")

# Standardize NCA output
scaler_nca = StandardScaler()
X_scaled_nca = scaler_nca.fit_transform(X_nca_transformed)

# Stratified split to avoid single-class issue
X_tr, X_te, y_tr, y_te = train_test_split(
    X_scaled_nca, y_encoded_labels, test_size=0.2, random_state=42, stratify=y_encoded_labels
)

# Hyperparameter grids
param_grid_dict = {
    "LogisticRegression": {'C': [0.01, 0.1, 1, 10], 'solver': ['lbfgs', 'liblinear']},
    "SVM": {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    "RandomForest": {'n_estimators': [100], 'max_depth': [None, 5, 10]},
    "NaiveBayes": {},
    "DecisionTree": {'max_depth': [None, 5, 10]},
    "AdaBoost": {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 1.0]},
    "MLP": {'hidden_layer_sizes': [(100,), (100, 50)], 'activation': ['relu'], 'learning_rate': ['constant']},
    "KNN": {'n_neighbors': list(range(1, 11))}
}
if xgb_enabled:
    param_grid_dict["XGBoost"] = {'n_estimators': [100], 'max_depth': [3, 5], 'learning_rate': [0.1]}

# Define base models
model_dict = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42),
    "KNN": KNeighborsClassifier()
}
if xgb_enabled:
    model_dict["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Train & tune models
tuned_models = {}
for model_name, model in model_dict.items():
    print(f"🔧 Tuning {model_name}...")
    try:
        if param_grid_dict[model_name]:
            search = GridSearchCV(model, param_grid_dict[model_name], cv=5, scoring='accuracy', n_jobs=-1)
            search.fit(X_tr, y_tr)
            tuned_models[model_name] = search.best_estimator_
            print(f"Best parameters: {search.best_params_}")
        else:
            model.fit(X_tr, y_tr)
            tuned_models[model_name] = model
            print("No hyperparameters to tune.")
    except ValueError as e:
        print(f"❌ Error training {model_name}: {e}")

# Add stacking model
stacking_estimators = [(name.lower(), est) for name, est in tuned_models.items()]
stack_model = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)
tuned_models["StackingEnsemble"] = stack_model

# Evaluate all models
print("\nModel Evaluation:")
for name, model in tuned_models.items():
    try:
        model.fit(X_tr, y_tr)
        y_pred_test = model.predict(X_te)
        acc_train = accuracy_score(y_tr, model.predict(X_tr))
        acc_test = accuracy_score(y_te, y_pred_test)

        print(f"\n{name}")
        print(f"Train Accuracy: {acc_train:.4f}")
        print(f"Test Accuracy:  {acc_test:.4f}")
        print("Classification Report:")
        print(classification_report(y_te, y_pred_test, target_names=[str(cls) for cls in le.classes_]))
        print("-" * 60)
    except Exception as e:
        print(f"{name} evaluation failed: {e}")

# LIME explainability
print("\nLIME Explanation (on one sample):")
lime_explainer = LimeTabularExplainer(
    training_data=X_features,
    feature_names=[f"f_{i}" for i in range(X_features.shape[1])],
    class_names=list(le.classes_),
    mode='classification',
    discretize_continuous=False
)
sample_instance = X_features[0]
for name, model in tuned_models.items():
    if name == "StackingEnsemble":
        print(f"{name}: Skipping LIME")
        continue
    pipeline = make_pipeline(StandardScaler(), NeighborhoodComponentsAnalysis(n_components=2, random_state=42), model)
    pipeline.fit(X_features, y_encoded_labels)
    prediction = pipeline.predict([sample_instance])[0]
    pred_class = le.inverse_transform([prediction])[0]
    print(f"{name} ➜ Predicted class: {pred_class}")
    try:
        exp = lime_explainer.explain_instance(
            data_row=sample_instance,
            predict_fn=pipeline.predict_proba,
            num_features=10,
            num_samples=5000
        )
        exp.save_to_file(f"lime_{name}_explanation.html")
        fig = exp.as_pyplot_figure()
        plt.tight_layout()
        fig.savefig(f"lime_{name}_explanation.png")
        plt.close(fig)
    except Exception as ex:
        print(f"Failed LIME for {name}: {ex}")
    print("-" * 60)



Top 10 features for NCA Component 1:
Feature 34: Weight = -18.5722
Feature 749: Weight = 16.2427
Feature 266: Weight = 14.4001
Feature 633: Weight = -14.1081
Feature 425: Weight = -12.8418
Feature 349: Weight = 12.7638
Feature 13: Weight = 12.5606
Feature 122: Weight = 12.2354
Feature 110: Weight = -12.1251
Feature 471: Weight = -12.0630

Top 10 features for NCA Component 2:
Feature 555: Weight = -18.8237
Feature 301: Weight = -16.0748
Feature 597: Weight = 13.4197
Feature 363: Weight = 12.7880
Feature 264: Weight = -12.7670
Feature 753: Weight = 12.5523
Feature 11: Weight = 12.4286
Feature 746: Weight = 11.1177
Feature 86: Weight = 11.0924
Feature 182: Weight = 11.0040
🔧 Tuning LogisticRegression...
Best parameters: {'C': 0.1, 'solver': 'lbfgs'}
🔧 Tuning SVM...
Best parameters: {'C': 10, 'kernel': 'rbf'}
🔧 Tuning RandomForest...
Best parameters: {'max_depth': 10, 'n_estimators': 100}
🔧 Tuning NaiveBayes...
No hyperparameters to tune.
🔧 Tuning DecisionTree...
Best parameters: {'max_de

c:\Users\vinee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Best parameters: {'learning_rate': 0.1, 'n_estimators': 100}
🔧 Tuning MLP...
Best parameters: {'activation': 'relu', 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant'}
🔧 Tuning KNN...
Best parameters: {'n_neighbors': 9}
🔧 Tuning XGBoost...


c:\Users\vinee\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:10:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

Model Evaluation:

LogisticRegression
Train Accuracy: 0.7876
Test Accuracy:  0.7763
Classification Report:
              precision    recall  f1-score   support

           1       0.81      0.98      0.88       345
           2       0.78      0.66      0.72       346
           3       0.74      0.69      0.71       346

    accuracy                           0.78      1037
   macro avg       0.77      0.78      0.77      1037
weighted avg       0.77      0.78      0.77      1037

------------------------------------------------------------

SVM
Train Accuracy: 0.8601
Test Accuracy:  0.8544
Classification Report:
              precision    recall  f1-score   support

           1       0.90      0.97      0.94       345
           2       0.83      0.81      0.82       346
           3       0.83      0.78      0.80       346

    accuracy                           0.85      1037
   macro avg       0.85    

...


StackingEnsemble
Train Accuracy: 0.8982
Test Accuracy:  0.8679
Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.97      0.95       345
           2       0.85      0.80      0.83       346
           3       0.82      0.83      0.83       346

    accuracy                           0.87      1037
   macro avg       0.87      0.87      0.87      1037
weighted avg       0.87      0.87      0.87      1037

------------------------------------------------------------

LIME Explanation (on one sample):
LogisticRegression ➜ Predicted class: 3
------------------------------------------------------------
SVM ➜ Predicted class: 3
------------------------------------------------------------
RandomForest ➜ Predicted class: 3
------------------------------------------------------------
NaiveBayes ➜ Predicted class: 3
------------------------------------------------------------
DecisionTree ➜ Predicted class: 3
---------------------------

c:\Users\vinee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost ➜ Predicted class: 3
------------------------------------------------------------
MLP ➜ Predicted class: 3
------------------------------------------------------------
KNN ➜ Predicted class: 3
------------------------------------------------------------


c:\Users\vinee\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:26:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost ➜ Predicted class: 3
------------------------------------------------------------
StackingEnsemble: Skipping LIME
